In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,mean_absolute_percentage_error

In [2]:
end = pd.datetime.now().date() - pd.Timedelta(days = 1*252)
start = end - pd.Timedelta(days = ((12*252)+30))

<ipython-input-2-2ca4f16373c0>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  end = pd.datetime.now().date() - pd.Timedelta(days = 1*252)


In [3]:
class backtest_ML:
    
    def __init__(self,ticker,start,end):
          
        self.ticker = ticker
        self.start = start
        self.end = end
    
        self.fetch_data()
        self.process_data()
        self.train_test_split()
        self.model()
        self.predict()
        self.model_performance()
    
    def fetch_data(self):
        self.df = yf.download(self.ticker,self.start,self.end)
        self.df1=yf.download("^VIX",self.start,self.end)
    def process_data(self):
        
        self.df['Avg Price'] = talib.AVGPRICE(self.df['Open'].shift(1),self.df['High'].shift(1),self.df['Low'].shift(1),self.df['Adj Close'].shift(1))
        self.df['Gap prev close'] = np.log((self.df['Open'])/(self.df['Adj Close'].shift(1)))
        self.df['Gap prev Avg price'] = np.log((self.df['Open'])/(self.df['Avg Price'].shift(1)))
        self.df['MA_1'] = self.df['Adj Close'].shift(1).rolling(window=5,center=False).mean() 
        self.df['VIX'] = self.df1['Open'] 
        self.df['RSI'] =talib.RSI(self.df['Adj Close'].shift(1),timeperiod = 5)
        self.df['Momentum'] =talib.MOM(self.df['Adj Close'].shift(1),timeperiod=5)
        self.df['MFI'] = talib.MFI(self.df['High'].shift(1),self.df['Low'],self.df['Adj Close'],self.df['Volume'],timeperiod=5)
        self.df['OBV'] = talib.OBV(self.df['Volume'].shift(1),self.df['Adj Close'].shift(1))
        self.df['ATR'] = talib.ATR(self.df['High'].shift(1),self.df['Low'].shift(1),self.df['Adj Close'].shift(1),timeperiod=5)
        self.df['HB Signal'] = talib.HT_DCPERIOD(self.df['Adj Close'])
        self.df['Correl'] = talib.CORREL(self.df['High'].shift(1),self.df['Low'].shift(1),timeperiod=5) 
        self.df['Daily Returns'] =np.log(self.df['Adj Close']/self.df['Adj Close'].shift(1))
        self.df['Three Line Strike'] = talib.CDL3LINESTRIKE(self.df['Open'].shift(1),self.df['High'].shift(1),self.df['Low'].shift(1),self.df['Adj Close'].shift(1))
        self.df = self.df[(self.df['Daily Returns'] < 0.02) & (self.df['Daily Returns'] > -0.02)]
        self.df.dropna(inplace=True)
        
    def train_test_split(self):
        self.X = self.df.iloc[:,[6,7,8,9,10,11,12,13,14,15,16,17,19]]
        self.y = self.df['Adj Close']
        self.X_train,self.X_test,self.y_train,self.y_test = train_test_split(self.X,self.y,test_size=0.3,random_state=200)
        
    def model(self):
        self.mod = GradientBoostingRegressor(n_estimators=160)
        self.mod.fit(self.X_train,self.y_train)
        
    def predict(self):
        self.y_pred_test = self.mod.predict(self.X_test)
        self.y_pred_train = self.mod.predict(self.X_train)
        
    def model_performance(self):
        
        Score_Train = self.mod.score(self.X_train,self.y_train)
        Score_Test = self.mod.score(self.X_test,self.y_test)
        R2_score_Train = r2_score(self.y_train,self.y_pred_train)
        R2_score_Test = r2_score(self.y_test,self.y_pred_test)
        MAE_Train = mean_absolute_error(self.y_train,self.y_pred_train)
        MAE_Test = mean_absolute_error(self.y_test,self.y_pred_test)
        MSE_Train = mean_squared_error(self.y_train,self.y_pred_train)
        MSE_Test = mean_squared_error(self.y_test,self.y_pred_test)
        MAPE_Train = mean_absolute_percentage_error(self.y_train,self.y_pred_train)
        MAPE_Test = mean_absolute_percentage_error(self.y_test,self.y_pred_test)
        
        return (Score_Train,Score_Test,R2_score_Train,R2_score_Test,MAE_Train,MAE_Test,MSE_Train,
        MSE_Test,MAPE_Train,MAPE_Test)

In [4]:

a=[]

Nifty_backtest_ML = backtest_ML("TCS.NS",start,end)
a.append(Nifty_backtest_ML.model_performance())
Score_Train = [a[0] for a in a]
Score_Test = [a[1] for a in a]
R2_score_Train = [a[2] for a in a]
R2_score_Test = [a[3] for a in a]
MAE_Train = [a[4] for a in a]
MAE_Test = [a[5] for a in a]
MSE_Train = [a[6] for a in a]
MSE_Test = [a[7] for a in a]
MAPE_Train = [a[8] for a in a]
MAPE_Test = [a[9] for a in a]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
Results = pd.DataFrame({'Score_Train':Score_Train,'Score_Test':Score_Test,'R2_score_Train':R2_score_Train,'R2_score_Test'
                       :R2_score_Test,'MAE_Train':MAE_Train,'MAE_Test':MAE_Test,'MSE_Train':MSE_Train,
                       'MSE_Test':MSE_Test,'MAPE_Train':MAPE_Train,'MAPE_Test':MAPE_Test})

Results

,Score_Train,Score_Test,R2_score_Train,R2_score_Test,MAE_Train,MAE_Test,MSE_Train,MSE_Test,MAPE_Train,MAPE_Test
0,0.999773,0.998965,0.999773,0.998965,7.243105,14.152401,90.632221,402.034511,0.005156,0.009612


In [6]:
Nifty_backtest_ML.mod.feature_importances_

array([6.79654165e-01, 2.19476933e-03, 1.82823434e-04, 2.20085228e-01,
       1.42648352e-05, 5.71657677e-05, 1.62144177e-05, 1.04012724e-04,
       9.75735826e-02, 8.14997411e-05, 1.44028540e-05, 2.18718540e-05,
       0.00000000e+00])

In [7]:
feature_imp=pd.Series(Nifty_backtest_ML.mod.feature_importances_,index=Nifty_backtest_ML.X.columns)
feature_imp.sort_values(ascending=False)

Avg Price             0.679654
MA_1                  0.220085
OBV                   0.097574
Gap prev close        0.002195
Gap prev Avg price    0.000183
MFI                   0.000104
ATR                   0.000081
RSI                   0.000057
Correl                0.000022
Momentum              0.000016
HB Signal             0.000014
VIX                   0.000014
Three Line Strike     0.000000
dtype: float64

In [8]:
## Features in back up

self.df['Prev day'] = np.log((self.df['Adj Close'].shift(1))/(self.df['Adj Close'].shift(2)))
self.df['2 Days Low'] = np.where((self.df['Daily Returns'].shift(1) < 0) & (self.df['Daily Returns'].shift(2) < 0),1,0)
self.df['2 Days High'] = np.where((self.df['Daily Returns'].shift(1) > 0) & (self.df['Daily Returns'].shift(2) > 0),1,0)
self.df['Stdev_1']=self.df['Adj Close'].shift(1).rolling(window=3,center=False).std()
self.df['Avg Price'] = talib.AVGPRICE(self.df['Open'].shift(1),self.df['High'].shift(1),self.df['Low'].shift(1),self.df['Adj Close'].shift(1))
        

NameError: name 'self' is not defined

In [ ]:
Nifty_backtest_ML.X_train.corr()